Laburanding en ver de diferenciar una senial de noise de una de FM, para luego extender a varias mas.

Data logged using:
uhd_ -f -b 

**Se ajusto todo para debuggear porque no sale de .5 accuracy, ver problema.**

In [226]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dropout, Conv1D, Dense, Activation, MaxPooling1D, Input
from tensorflow.keras.utils import Sequence as Sequence
import numpy as np
from google.colab import drive
from datetime import datetime
import ast

sample_rate = 100000
signals_count = 2
epochs = 2000
batch_size = 128
capture_size = 2048

# Debug
#train_size = 16
#test_size = 4
#capture_size = 8
# Test
#train_size = 50
#test_size = 5
# Real
train_size = 1200
test_size = 200

dir_base_model = "/content/gdrive/My Drive/DeepLearning/sigid/test-"
print(tf.__version__)

2.1.0


In [227]:
drive.mount('/content/gdrive')

# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('i', 'f4'), ('q', 'f4')])
# Load noisy data
data = np.fromfile(dir_base_model + "nothing.iq", dtype=dt)
dfa = pd.DataFrame(data)
dfa["label"] = "BLANK"
npArray = np.array(range(len(dfa)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfa["time"] = npArray / sample_rate
# Load FM data
data = np.fromfile(dir_base_model + "999.iq", dtype=dt)
dfb = pd.DataFrame(data)
dfb["label"] = "FM"
npArray = np.array(range(len(dfb)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfb["time"] = npArray / sample_rate

frames = [dfa, dfb]
result = pd.concat(frames)
#print(result)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train = []
test = []

# Build datasets for train 
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfa[ii:ii + capture_size].values)
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfb[ii:ii + capture_size].values)

# Build datasets for test 
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfa[ii:ii + capture_size].values)
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfb[ii:ii + capture_size].values)

In [229]:
print('Capture size dfa BLANK: ' + str(len(dfa.values)))
print('Capture size dfb FM: ' + str(len(dfb.values)))

Capture size dfa BLANK: 3059712
Capture size dfb FM: 2646528


In [230]:
# Plot some sample data to check iq files
import plotly.express as px
fig = px.line(dfa[1:capture_size], x="time", y="i", labels={'X':'I'}, width=1000, height=300)
fig.show()
fig = px.line(dfb[1:capture_size], x="time", y="i", labels={'X':'I'}, width=1000, height=300)
fig.show()

In [231]:
def get_signal_dataframes(df):
  interleave = 2
  dx = []
  dy = []
  tx = []
  ty = []

  # Generate train dataset 
  dfax = df[['i', 'q']] 
  dfay = df[['label']] 
  for i in range(0, train_size):
    newx = dfax[interleave * i:interleave * i + capture_size]
    dx.append(newx)    
    newy = dfay[interleave * i:interleave * i + capture_size]
    dy.append(newy)

  # Offset to separate train from test datain df
  offset = train_size * interleave + capture_size

  print("Offset " + str(offset))

  # Generate test dataset 
  for i in range(0, test_size):
    newx = dfax[interleave * i + offset:interleave * i + capture_size + offset]
    print(newx)
    tx.append(newx)
    newy = dfay[interleave * i + offset:interleave * i + capture_size + offset]
    print(newy)
    ty.append(newy)

  # Convert lists to dataframes
  train_x = pd.DataFrame(dx) 
  train_y = pd.DataFrame(dy) 
  test_x = pd.DataFrame(tx) 
  test_y = pd.DataFrame(ty) 
  return train_x, train_y, test_x, test_y

def add_dataframe_for_signal(df, train_x, train_y, test_x, test_y):
  x, y, tx, ty = get_signal_dataframes(df)
  train_x.append(x)
  train_y.append(y)
  test_x.append(tx)
  test_y.append(ty)

train_x = []
train_y = []
test_x = []
test_y = []
add_dataframe_for_signal(dfa, train_x, train_y, test_x, test_y)
add_dataframe_for_signal(dfb, train_x, train_y, test_x, test_y)

# Build dataframes 
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)
test_x = pd.concat(test_x)
test_y = pd.concat(test_y)


print("Initial X " + str(len(train_x.values)))
for i in range(len(train_x.values)):
  print(train_x.values[i][0].values[0])
#print(train_x.values)

print("")
print("Initial Y " + str(len(train_y.values)))
for i in range(len(train_y.values)):
  print(train_y.values[i][0].values[0])
#print(train_y.values)

permutations = np.random.permutation(len(train_y))
train_x = train_x.iloc[permutations]
train_y = train_y.iloc[permutations]

# Da filas 32
print("")
print("Final X " + str(len(train_x.values)))
for i in range(len(train_x.values)):
   print(train_x.values[i][0].values[0])
#print(train_x.values)

print("")
print("Final Y " + str(len(train_y.values)))
for i in range(len(train_y.values)):
  print(train_y.values[i][0].values[0])
#print(train_y.values)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
3377    FM
3378    FM
3379    FM
3380    FM
...    ...
5419    FM
5420    FM
5421    FM
5422    FM
5423    FM

[2048 rows x 1 columns]
             i         q
3378  0.000061 -0.000092
3379 -0.000092  0.000031
3380  0.000000  0.000000
3381 -0.000061 -0.000061
3382  0.000000  0.000031
...        ...       ...
5421 -0.000092  0.000031
5422  0.000031  0.000122
5423 -0.000031  0.000092
5424  0.000000  0.000031
5425  0.000031 -0.000031

[2048 rows x 2 columns]
     label
3378    FM
3379    FM
3380    FM
3381    FM
3382    FM
...    ...
5421    FM
5422    FM
5423    FM
5424    FM
5425    FM

[2048 rows x 1 columns]
             i         q
3380  0.000000  0.000000
3381 -0.000061 -0.000061
3382  0.000000  0.000031
3383  0.000000 -0.000061
3384 -0.000031  0.000031
...        ...       ...
5423 -0.000031  0.000092
5424  0.000000  0.000031
5425  0.000031 -0.000031
5426 -0.000031  0.000031
5427 -0.000031  0.000031

[2048 rows x 2 

In [232]:
print("Initial X " + str(len(test_x.values)))
print(test_x.values)
print("")
print("Initial Y " + str(len(test_y.values)))
print(test_y.values)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[2048 rows x 1 columns]]
 [      label
3134  BLANK
3135  BLANK
3136  BLANK
3137  BLANK
3138  BLANK
...     ...
5177  BLANK
5178  BLANK
5179  BLANK
5180  BLANK
5181  BLANK

[2048 rows x 1 columns]]
 [      label
3136  BLANK
3137  BLANK
3138  BLANK
3139  BLANK
3140  BLANK
...     ...
5179  BLANK
5180  BLANK
5181  BLANK
5182  BLANK
5183  BLANK

[2048 rows x 1 columns]]
 [      label
3138  BLANK
3139  BLANK
3140  BLANK
3141  BLANK
3142  BLANK
...     ...
5181  BLANK
5182  BLANK
5183  BLANK
5184  BLANK
5185  BLANK

[2048 rows x 1 columns]]
 [      label
3140  BLANK
3141  BLANK
3142  BLANK
3143  BLANK
3144  BLANK
...     ...
5183  BLANK
5184  BLANK
5185  BLANK
5186  BLANK
5187  BLANK

[2048 rows x 1 columns]]
 [      label
3142  BLANK
3143  BLANK
3144  BLANK
3145  BLANK
3146  BLANK
...     ...
5185  BLANK
5186  BLANK
5187  BLANK
5188  BLANK
5189  BLANK

[2048 rows x 1 columns]]
 [      label
3144  BLANK
3145  BLANK
3146  BLAN

In [0]:
def recreate_np_array_x(d, size):
  new = np.ndarray(shape=(signals_count * size, 2, capture_size), dtype=float)
  # 100 i
  for i in range(len(d.values)):
    # 2048 j
    for j in range(len((d.values)[i])):
        new[i][0][j] = ((d.values)[i])[0].values[j][0]
        new[i][1][j] = ((d.values)[i])[0].values[j][1]
  return new

def recreate_np_array_y(d, size):
  new = np.ndarray(shape=(signals_count * size), dtype=np.int16)
  # 100 i
  for i in range(len(d.values)):
    result = ((d.values)[i])[0].values[0][0]
    if (result == 'FM'):
      new[i] = 1
    else:
      new[i] = 0
  return new

train_x = recreate_np_array_x(train_x, train_size)
train_y = recreate_np_array_y(train_y, train_size)
test_x = recreate_np_array_x(test_x, test_size)
test_y = recreate_np_array_y(test_y, test_size)

**Aca deberia poder poner el input como yo quiero sin tener que tocar el vector.**

In [0]:
# Real model to test: .805 accuracay
def model_a():
  # Real model to test
  model = Sequential()
  model.add(Conv1D(filters=10, kernel_size=16, strides=1, padding='same', activation='relu', input_shape=(2, capture_size)))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=12, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 2))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=6, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 4))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Conv1D(filters=10, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(6, int(capture_size / 8))))
  model.add(MaxPooling1D(pool_size=10, strides=2, padding='same'))
  model.add(Dense(2, activation='softmax'))
  #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #model.compile(loss = "categorical_crossentropy", optimizer = "adam")
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "sgd")
  return model


In [0]:
# Tensorboard test
from datetime import datetime
logdir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [0]:
# #TODO: Contar los distintos labels de los df
model = model_a()
model.summary()

# Train
model.fit(x = train_x, y = train_y, batch_size=batch_size, verbose=1, epochs=epochs, callbacks=[tensorboard_callback])
#model.fit(x = train_x, y = train_y, batch_size=batch_size, verbose=1, validation_data=(test_x, test_y), validation_steps=validation_steps, epochs=epochs, callbacks=[tensorboard_callback])


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_60 (Conv1D)           (None, 2, 10)             327690    
_________________________________________________________________
max_pooling1d_60 (MaxPooling (None, 1, 10)             0         
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 1, 10)             1210      
_________________________________________________________________
max_pooling1d_61 (MaxPooling (None, 1, 10)             0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 1, 10)             610       
_________________________________________________________________
max_pooling1d_62 (MaxPooling (None, 1, 10)             0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 1, 10)           

In [0]:
def read_saved_info():
  try:
      file = open(dir_base_model + 'best-noise-vs-fm-info.txt', 'rt')
      value = file.read()
      file.close()
      old_evaluation_result = ast.literal_eval(value)
      return old_evaluation_result
  except FileNotFoundError:
      return None

In [0]:
# Evaluate model
evaluation_result = model.evaluate(test_x, test_y, verbose=1)
print("Current loss is: " + str(evaluation_result))

old_evaluation_result = read_saved_info()
exists = False
print("Old loss is: " + str(old_evaluation_result))
if (old_evaluation_result != None):
  exists = True

if (exists == False or (evaluation_result[0] < old_evaluation_result[0])):
  print("Writting new object...")
  file = open(dir_base_model + 'best-noise-vs-fm-info.txt', 'wt')
  file.write(str(evaluation_result))
  file.close()
  model.save(dir_base_model + 'best-noise-vs-fm-info.model')

In [0]:
# Draw training data
# Draw confusion matrix
# https://plot.ly/~francoisp/50/confusion-matrix/#code


In [0]:
# Prediccion del mejor modelo
predictions = model.predict(test_x[:3])